# Finite difference - optioner

In [389]:
import numpy as np
import random
import math
import time
from collections import defaultdict

In [420]:
S0 = 100
K = 100
T = 1
r = 0.07
sigma = 0.2
n_simulations = 100000
n_steps = 252

## Asian option

### Finite difference ved Monte Carlo

In [410]:
# Finite Difference metoden
def finite_difference_MC_asian(S0, K, T, r, sigma, n_simulations, n_steps, h=0.0001):
    # Udvikling i prisen på det underliggende aktiv
    Z = np.random.normal(0,1,size=(n_simulations, n_steps))
    
    #Definer prisfunktionen for en Asiatisk option
    def MC_Payoff(S0, K, T, r, sigma, n_simulations, n_steps):
        dt = T/n_steps
        S = np.zeros((n_simulations, n_steps))
        S[:,0] += S0
        for i in range(1, n_steps):
            S[:,i] += S[:,i-1]*np.exp((r-(sigma**2)/2)*dt+sigma*np.sqrt(dt)*Z[:,i])

        def Geo_Mean_Overflow(iterable):
            temp = np.log(iterable)
            return np.exp(temp.mean()) 

        avg = [Geo_Mean_Overflow(S[i,:]) for i in range(len(S))]

        c = [avg[i] - K for i in range(len(avg))]
        c = [np.maximum(c[i], 0) for i in range(len(avg))]

        payoff_call = np.mean(c)*np.exp(-r*T)
    
        return payoff_call
    
    adjoints = defaultdict(int)
    
    adjoints["S_"] = (MC_Payoff(S0+h, K, T, r, sigma, n_simulations, n_steps) - MC_Payoff(S0, K, T, r, sigma, n_simulations, n_steps)) / h
    adjoints["K_"] = (MC_Payoff(S0, K+h, T, r, sigma, n_simulations, n_steps) - MC_Payoff(S0, K, T, r, sigma, n_simulations, n_steps)) / h
    adjoints["T_"] = (MC_Payoff(S0, K, T+h, r, sigma, n_simulations, n_steps) - MC_Payoff(S0, K, T, r, sigma, n_simulations, n_steps)) / h
    adjoints["r_"] = (MC_Payoff(S0, K, T, r+h, sigma, n_simulations, n_steps) - MC_Payoff(S0, K, T, r, sigma, n_simulations, n_steps)) / h
    adjoints["sigma_"] = (MC_Payoff(S0, K, T, r, sigma+h, n_simulations, n_steps) - MC_Payoff(S0, K, T, r, sigma, n_simulations, n_steps)) / h
    return adjoints

In [411]:
start_time = time.time()

print(finite_difference_MC_asian(S0, K, T, r, sigma, n_simulations, n_steps))

print("--- %s seconds ---" % (time.time() - start_time))

defaultdict(<class 'int'>, {'S_': 0.6047984527679517, 'K_': -0.5448350047299044, 'T_': 3.5779297203841764, 'r_': 24.124218841317102, 'sigma_': 18.893485758946227})
--- 26.861389875411987 seconds ---


### Finite difference med Black Scholes formlen 

In [415]:
def finite_difference_BS(S0, K, T, r, sigma, h):
    
    def BS_Price_Asian_Options(S0, K, T, r, sigma):
    
        b = (1/2) * (r - (1/2) * ((sigma / 1.7320508075688772)**2))
    
        d1 = (np.log(S0 / K) + (b + (1/2) * ((sigma / 1.7320508075688772)**2) ) * T)/ ((sigma / 1.7320508075688772) * np.sqrt(T))
    
        d2 = d1 - (sigma / 1.7320508075688772)*np.sqrt(T)
        p= S0 * np.exp((b - r) * T) * norm.cdf(d1) - K * np.exp(-(1) * r * T) * norm.cdf(d2)
    
        return p
    adjoints = defaultdict(int)
    
    adjoints["S_"] = (BS_Price_Asian_Options(S0 +h, K, T, r, sigma) - BS_Price_Asian_Options(S0, K, T, r, sigma)) / h
    adjoints["K_"] = (BS_Price_Asian_Options(S0 , K+h, T, r, sigma) - BS_Price_Asian_Options(S0, K, T, r, sigma)) / h
    adjoints["T_"] = (BS_Price_Asian_Options(S0, K, T+h, r, sigma) - BS_Price_Asian_Options(S0, K, T, r, sigma)) / h
    adjoints["r_"] = (BS_Price_Asian_Options(S0 , K, T, r+h, sigma) - BS_Price_Asian_Options(S0, K, T, r, sigma)) / h
    adjoints["sigma_"] = (BS_Price_Asian_Options(S0 , K, T, r, sigma+h) - BS_Price_Asian_Options(S0, K, T, r, sigma)) / h
    return adjoints

In [416]:
start_time = time.time()
print(finite_difference_BS(S0, K, T, r, sigma, 0.001))
print("--- %s seconds ---" % (time.time() - start_time))

defaultdict(<class 'int'>, {'S_': 0.6063674649752215, 'K_': -0.5460905548773098, 'T_': 3.5955351920833323, 'r_': 24.312613394428695, 'sigma_': 18.95732800231542})
--- 0.0042569637298583984 seconds ---


## European option

### Finite Difference for Monte Carlo

In [425]:
def finite_difference_MC_euro(S0, K, T, r, sigma, n_simulations, n_steps, h=0.0001):
    # Udvikling i prisen på det underliggende aktiv
    Z = np.random.normal(0,1,size=(n_simulations, n_steps))
 
    def Price_European_Option(S0, K, T, r,sigma, n_simulations, n_steps, h):
        dt = T/n_steps
        S = np.zeros((n_simulations, n_steps))
        S[:,0] += S0
        for i in range(1, n_steps):
            S[:,i] += S[:,i-1]*np.exp((r-(sigma**2)/2)*dt+sigma*np.sqrt(dt)*Z[:,i])
        
        # Beregning af payoff for call
        c = S[:,-1] + h - K
        for i in range(len(c)):
            c[i] = np.maximum(c[i], 0)

        # Gennemsnitligt payoff tilbagediskonteret
        payoff_call = np.mean(c)*np.exp(-r*T)
   
        return payoff_call

    adjoints = defaultdict(int)
    
    adjoints["S_"] = (Price_European_Option(S0 +h, K, T, r, sigma,n_simulations, n_steps, h) - Price_European_Option(S0, K, T, r, sigma,n_simulations, n_steps, h)) / h
    adjoints["K_"] = (Price_European_Option(S0 , K+h, T, r, sigma,n_simulations, n_steps, h) - Price_European_Option(S0, K, T, r, sigma,n_simulations, n_steps, h)) / h
    adjoints["T_"] = (Price_European_Option(S0, K, T+h, r, sigma,n_simulations, n_steps, h) - Price_European_Option(S0, K, T, r, sigma,n_simulations, n_steps, h)) / h
    adjoints["r_"] = (Price_European_Option(S0 , K, T, r+h, sigma,n_simulations, n_steps, h) - Price_European_Option(S0, K, T, r, sigma,n_simulations, n_steps, h)) / h
    adjoints["sigma_"] = (Price_European_Option(S0 , K, T, r, sigma+h,n_simulations, n_steps, h) - Price_European_Option(S0, K, T, r, sigma,n_simulations, n_steps, h)) / h
    return adjoints


In [426]:
start_time = time.time()

print(finite_difference_MC_euro(S0, K, T, r, sigma, n_simulations, n_steps))

print("--- %s seconds ---" % (time.time() - start_time))

defaultdict(<class 'int'>, {'S_': 0.6725752426461895, 'K_': -0.557850476905486, 'T_': 7.465620650268789, 'r_': 55.521628572705595, 'sigma_': 35.796309436886276})
--- 18.751221895217896 seconds ---


In [ ]:
def BS_Price_European_Options(S0, K, T, r, sigma): 
    d1 = (np.log(S0/K)+(r+1/2*(sigma**2))*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    euro_call = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    euro_put = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    
    return euro_call

In [430]:
def finite_difference_BS(S0, K, T, r, sigma, h):
    
    def BS_Price_European_Options(S0, K, T, r, sigma): 
        d1 = (np.log(S0/K)+(r+1/2*(sigma**2))*T)/(sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)

        euro_call = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
        euro_put = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)
    
        return euro_call
    
    adjoints = defaultdict(int)
    
    adjoints["S_"] = (BS_Price_European_Options(S0 +h, K, T, r, sigma) - BS_Price_European_Options(S0, K, T, r, sigma)) / h
    adjoints["K_"] = (BS_Price_European_Options(S0 , K+h, T, r, sigma) - BS_Price_European_Options(S0, K, T, r, sigma)) / h
    adjoints["T_"] = (BS_Price_European_Options(S0, K, T+h, r, sigma) - BS_Price_European_Options(S0, K, T, r, sigma)) / h
    adjoints["r_"] = (BS_Price_European_Options(S0 , K, T, r+h, sigma) - BS_Price_European_Options(S0, K, T, r, sigma)) / h
    adjoints["sigma_"] = (BS_Price_European_Options(S0 , K, T, r, sigma+h) - BS_Price_European_Options(S0, K, T, r, sigma)) / h
    
    return adjoints

In [433]:
start_time = time.time()
print(finite_difference_BS(S0, K, T, r, sigma, 0.001))
print("--- %s seconds ---" % (time.time() - start_time))

defaultdict(<class 'int'>, {'S_': 0.6736537927736208, 'K_': -0.5582210648285013, 'T_': 7.511817785889718, 'r_': 55.88513957093966, 'sigma_': 36.06278151150377})
--- 0.0070819854736328125 seconds ---


## Simple Barrier Up and In

### Finite difference ved Monte Carlo

In [471]:
S0 = 12.418
K = 13.55
T = 1/12
r = 0.01606
sigma = 0.4075
n_simulations = 10000000
n_steps = 1
B=17

In [472]:
def finite_difference_MC_Barrier(S0, K, T, r, sigma, B, n_simulations, n_steps, h=0.0001):
    # Udvikling i prisen på det underliggende aktiv
    timeline = np.arange(0,T,T/n_steps)
    Z = np.random.normal(0,1,size=(n_simulations, len(timeline)))
    
    def Price_Barrier_Option(S0, K, T, r,sigma, B, n_simulations, n_steps, h):
        dt = T/n_steps
        d1 = r - ( sigma **2) / 2
        d2 = d1 * dt + sigma * np.sqrt( dt ) * Z
        #S[:,0] += S0
        S = np.hstack ([ S0 * np.ones (( n_simulations , 1) ) , np . exp ( d2 ) ]) . cumprod ( axis = 1)

        ST = S[:,-1]
        P=np.maximum( ST - K, 0)
        P = P * 1/(1+np.exp(-K *(ST - B)))
        V = np.mean(np.exp ( - r * T ) * P)
         
        return V
    print(Price_Barrier_Option(S0, K, T, r,sigma, B, n_simulations, n_steps, h))
    adjoints = defaultdict(int)
    
    adjoints["S_"] = (Price_Barrier_Option(S0 +h, K, T, r, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["K_"] = (Price_Barrier_Option(S0 , K+h, T, r, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["T_"] = (Price_Barrier_Option(S0, K, T+h, r, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["r_"] = (Price_Barrier_Option(S0 , K, T, r+h, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["sigma_"] = (Price_Barrier_Option(S0 , K, T, r, sigma+h,B,n_simulations, n_steps, h) - Price_Barrier_Option(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    return adjoints


In [473]:
start_time = time.time()

print(finite_difference_MC_Barrier(S0, K, T, r, sigma,B, n_simulations, n_steps))

print("--- %s seconds ---" % (time.time() - start_time))

0.013591924863883111
defaultdict(<class 'int'>, {'S_': 0.028124316997675536, 'K_': -0.003378156505466634, 'T_': 0.6575909235474221, 'r_': 0.027971294608748754, 'sigma_': 0.2665709361583494})
--- 7.583154916763306 seconds ---


## Barrier Down and Out

### Finite difference ved Monte Carlo

In [494]:
S0 = 12.418
K = 13.55
T = 1/12
r = 0.01606
sigma = 0.4075
n_simulations = 10000000
n_steps = 10
B=12

In [495]:
def finite_difference_MC_Barrier_D(S0, K, T, r, sigma, B, n_simulations, n_steps, h=0.0001):
    # Udvikling i prisen på det underliggende aktiv
    timeline = np.arange(0,T,T/n_steps)
    Z = np.random.normal(0,1,size=(n_simulations, len(timeline)))
    
    def Price_Barrier_Option_D(S0, K, T, r,sigma, B, n_simulations, n_steps, h):
        dt = T/n_steps
        d1 = r - ( sigma **2) / 2
        d2 = d1 * dt + sigma * np.sqrt( dt ) * Z
        S = np.hstack ([ S0 * np.ones((n_simulations, 1) ) , np.exp( d2 ) ]).cumprod( axis = 1)

        max = np.maximum

        st = max(np.log(S[:,: -1]/B) , 0)


        stp = max( np.log(S[:,1:]/B ) , 0)

        p = np . exp ( -2 * st * stp / ( sigma **2 * dt ) )
        q = np . prod (1 - p , axis = 1)

        P = max (S [: , -1] - K , 0) * q

        V = np.mean(np.exp ( - r * T ) * P)
         
        return V
    print(Price_Barrier_Option_D(S0, K, T, r,sigma, B, n_simulations, n_steps, h))
    
    adjoints = defaultdict(int)
    
    adjoints["S_"] = (Price_Barrier_Option_D(S0 +h, K, T, r, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option_D(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["K_"] = (Price_Barrier_Option_D(S0 , K+h, T, r, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option_D(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["T_"] = (Price_Barrier_Option_D(S0, K, T+h, r, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option_D(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["r_"] = (Price_Barrier_Option_D(S0 , K, T, r+h, sigma,B,n_simulations, n_steps, h) - Price_Barrier_Option_D(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    adjoints["sigma_"] = (Price_Barrier_Option_D(S0 , K, T, r, sigma+h,B,n_simulations, n_steps, h) - Price_Barrier_Option_D(S0, K, T, r, sigma,B,n_simulations, n_steps, h)) / h
    return adjoints



In [496]:
start_time = time.time()

print(finite_difference_MC_Barrier_D(S0, K, T, r, sigma,B, n_simulations, n_steps))

print("--- %s seconds ---" % (time.time() - start_time))

0.13933584588139827
defaultdict(<class 'int'>, {'S_': 0.3469903625222259, 'K_': -0.127171392179104, 'T_': 1.285483894085948, 'r_': 0.17124386588268647, 'sigma_': 0.5124962681174461})
--- 200.0834550857544 seconds ---
